In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
import numpy as np
from scipy.sparse import vstack

def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out = [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        if n_sample == 0:
            n_sample = 1

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0)


def classwise_density_weighted_sampling_by_fraction(X_pool, y_pool, model, fraction=0.2, random_state=42):
    X_selected = []
    y_selected = []
    selected_indices = []

    classes = np.unique(y_pool)
    probs = model.predict_proba(X_pool)
    rng = np.random.RandomState(seed=random_state)

    # Initialize NearestNeighbors for density calculation
    nn = NearestNeighbors(n_neighbors=5)
    nn.fit(X_pool)

    for c in classes:
        idx_c = np.where(y_pool == c)[0]
        X_c = X_pool[idx_c]
        probs_c = probs[idx_c]

        n_c = len(idx_c)
        n_select = max(1, int(n_c * fraction))

        if n_c == 0:
            continue

        # Calculate uncertainty: 1 - max(probability)
        uncertainties = 1 - np.max(probs_c, axis=1)  # shape: (n_samples_in_class,)

        # Calculate density: inverse of sum of distances to k-nearest neighbors
        distances = nn.kneighbors(X_c, n_neighbors=5, return_distance=True)[0]
        density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)

        # Combine uncertainty and density
        scores = uncertainties * density

        # Select top-k samples with highest scores
        sorted_idx = np.argsort(scores)[-n_select:]
        top_k_idx = idx_c[sorted_idx]

        selected_indices.extend(top_k_idx)
        X_selected.append(X_pool[top_k_idx])
        y_selected.append(y_pool[top_k_idx])

    selected_indices = np.array(selected_indices)
    X_selected = vstack(X_selected).toarray() if len(X_selected) > 0 else np.array([])
    y_selected = np.concatenate(y_selected) if len(y_selected) > 0 else np.array([])

    # Remaining data
    remaining_idx = np.setdiff1d(np.arange(len(X_pool)), selected_indices)
    X_rest = X_pool[remaining_idx]
    y_rest = y_pool[remaining_idx]

    # Print selected samples' label distribution
    print(f"Selected samples (fraction={fraction}): {len(y_selected)}")
    print(f"Label distribution: {np.bincount(y_selected, minlength=11)}")

    return X_rest, X_selected, y_rest, y_selected


# Example usage
X_pool = x_test_adv_0_1  # Your data
y_pool = y_label_0_1     # Your labels (0 to 10)

# Initial stratified sampling
X_train_init, y_train_init = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)
print(f"Initial labeled set size: {len(y_train_init)}")
print(f"Initial label distribution: {np.bincount(y_train_init, minlength=11)}")

# Train initial model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_init, y_train_init)

_, X_10, _, y_10 = classwise_density_weighted_sampling_by_fraction(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.1, random_state=42
)

Initial labeled set size: 102570
Initial label distribution: [75542   161   700  1743   377 10542    74   736 12435   172    88]


/tmp/ipykernel_558/342120937.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_558/342120937.py:62: RuntimeWarning: invalid value encountered in multiply
  scores = uncertainties * density
/tmp/ipykernel_558/342120937.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_558/342120937.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_558/342120937.py:62: RuntimeWarning: invalid value encountered in multiply
  scores = uncertainties * density
/tmp/ipykernel_558/342120937.py:59: RuntimeWarning: divide by zero encountered in divide
  density = 1 / np.sum(distances, axis=1)  # shape: (n_samples_in_class,)
/tmp/ipykernel_558/342120937.py:59: RuntimeWarning: divide by zero encountered in divide
  density

Selected samples (fraction=0.2): 205144
Label distribution: [151085    323   1400   3486    755  21084    148   1473  24870    344
    176]


In [6]:
X_10.shape

(205144, 56)

In [7]:
# from xgboost import XGBClassifier

# print('xgb')
# xgb10 = XGBClassifier()
# xgb10.fit(X_10, y_10)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf10 = RandomForestClassifier()
rf10.fit(X_10, y_10)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt10 = DecisionTreeClassifier()
# dt10.fit(X_10, y_10)

xgb
RF
DT


DecisionTreeClassifier()

In [ ]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [8]:


# base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

#     y_pred = xgb10.predict(X)
#     print(y_pred.shape)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/UNSW_Input10/x_test_adv_{attack_name}_Def{class_num+1}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/UNSW_Input10/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [81228    80   599   572    58  6722     3 24443   195    71]
(81228, 56) (81228,)
Save 0 to baseline_Def1.npy
(80, 56) (80,)
Save 1 to baseline_Def2.npy
(599, 56) (599,)
Save 2 to baseline_Def3.npy
(572, 56) (572,)
Save 3 to baseline_Def4.npy
(58, 56) (58,)
Save 4 to baseline_Def5.npy
(6722, 56) (6722,)
Save 5 to baseline_Def6.npy
(3, 56) (3,)
Save 7 to baseline_Def8.npy
(24443, 56) (24443,)
Save 8 to baseline_Def9.npy
(195, 56) (195,)
Save 9 to baseline_Def10.npy
(71, 56) (71,)
Save 10 to baseline_Def11.npy
Execution Time: 0.323138 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [227649    108   2069   3805    340 107722     35     86     80     19]
(227649, 56) (227649,)
Save 0 to BIM_Def1.npy
(108, 56) (108,)
Save 1 to BIM_Def2.npy
(2069, 56) (2069,)
Save 2 to BIM_Def3.npy
(3805, 56) (3805,)
Save 3 to BIM_Def4.npy
(340, 56) (340,)
Save 4 to BIM_Def5.npy
(107722, 56) (107722,)
Save 5 to BIM_Def6.npy
(35, 56) (35,)
Save 7 to BIM_Def8.npy
(8

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf10.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_DensityWeighted/UNSW_Input10/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_DensityWeighted/UNSW_Input10/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [80377    86   582   680    83  8234     3 23729   135    62]
(80377, 56) (80377,)
Save 0 to baseline_Def1.npy
(86, 56) (86,)
Save 1 to baseline_Def2.npy
(582, 56) (582,)
Save 2 to baseline_Def3.npy
(680, 56) (680,)
Save 3 to baseline_Def4.npy
(83, 56) (83,)
Save 4 to baseline_Def5.npy
(8234, 56) (8234,)
Save 5 to baseline_Def6.npy
(3, 56) (3,)
Save 7 to baseline_Def8.npy
(23729, 56) (23729,)
Save 8 to baseline_Def9.npy
(135, 56) (135,)
Save 9 to baseline_Def10.npy
(62, 56) (62,)
Save 10 to baseline_Def11.npy
Execution Time: 1.723629 seconds
(341913,)
[ 0  1  2  3  4  5  8  9 10] [215263      4   1208    962     17 124236    200     10     13]
(215263, 56) (215263,)
Save 0 to BIM_Def1.npy
(4, 56) (4,)
Save 1 to BIM_Def2.npy
(1208, 56) (1208,)
Save 2 to BIM_Def3.npy
(962, 56) (962,)
Save 3 to BIM_Def4.npy
(17, 56) (17,)
Save 4 to BIM_Def5.npy
(124236, 56) (124236,)
Save 5 to BIM_Def6.npy
(200, 56) (200,)
Save 8 to BIM_Def9.npy
(10, 56) (10,)
Sav

In [10]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

#     y_pred = dt10.predict(X)
#     print(y_pred.shape)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning_DensityWeighted/UNSW_Input10/x_test_adv_{attack_name}_Def{class_num+1}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning_DensityWeighted/UNSW_Input10/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [79326   184   794   592   204  9580     4 23007   164   116]
(79326, 56) (79326,)
Save 0 to baseline_Def1.npy
(184, 56) (184,)
Save 1 to baseline_Def2.npy
(794, 56) (794,)
Save 2 to baseline_Def3.npy
(592, 56) (592,)
Save 3 to baseline_Def4.npy
(204, 56) (204,)
Save 4 to baseline_Def5.npy
(9580, 56) (9580,)
Save 5 to baseline_Def6.npy
(4, 56) (4,)
Save 7 to baseline_Def8.npy
(23007, 56) (23007,)
Save 8 to baseline_Def9.npy
(164, 56) (164,)
Save 9 to baseline_Def10.npy
(116, 56) (116,)
Save 10 to baseline_Def11.npy
Execution Time: 0.096455 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [177933    404  25768  17395    518 113993   4551    475    424    452]
(177933, 56) (177933,)
Save 0 to BIM_Def1.npy
(404, 56) (404,)
Save 1 to BIM_Def2.npy
(25768, 56) (25768,)
Save 2 to BIM_Def3.npy
(17395, 56) (17395,)
Save 3 to BIM_Def4.npy
(518, 56) (518,)
Save 4 to BIM_Def5.npy
(113993, 56) (113993,)
Save 5 to BIM_Def6.npy
(4551, 56) (4551,)
Save 7 to B

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}10")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_ActiveLearning_DensityWeighted/UNSW_Input10/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input10_ActiveLearning_DensityWeighted.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB20,81228,80,599,572,58,6722,0,3,24443,195,71,113971
1,XGB20,227649,108,2069,3805,340,107722,0,35,86,80,19,341913
2,XGB20,177133,36,2223,3656,111,158562,0,25,90,65,12,341913
3,XGB20,227649,108,2069,3805,340,107722,0,35,86,80,19,341913
4,XGB20,146934,9,1402,37,33,181427,389,11382,294,0,6,341913
5,XGB20,225203,208,1453,1668,396,112290,0,9,174,329,183,341913
6,XGB20,232886,230,2838,2958,176,28991,0,9,73066,556,203,341913
7,XGB20,242712,26,23,705,26,91125,0,1,6886,349,60,341913
8,XGB20,221712,150,3126,1687,64,114929,0,1,170,57,17,341913
9,XGB20,203442,43,3162,3096,288,131623,0,0,184,59,16,341913
